In [ ]:
!pip install riotwatcher
!pip install playwright
!pip install --upgrade requests

In [ ]:
import os
import time
import re
from datetime import datetime, timedelta
import playwright
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
from riotwatcher import *
import pandas as pd
import numpy as np
import requests
import logging

In [ ]:
file_path='' #insert the file path where 'League of Legends_final_RANKED_SOLO_matches.xlsx' file is

df = pd.read_excel(file_path, sheet_name='Sheet1')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#file_path = '/content/drive/My Drive/Colab Notebooks/League of Legends_final_RANKED_SOLO_matches.xlsx'
#df = pd.read_excel(file_path)

In [ ]:
# Riot API Key (need to have an account in https://developer.riotgames.com/ and then check the API key)
api_key = 'XXXXXXXX' # insert API key
lol_watcher = LolWatcher(api_key)

In [ ]:
# Function to get the win-loss ratio for a given row (only from 2022 until today, no 2021 detailed matches info)
def get_win_loss_ratios(row):
    try:
        # Define the years for which you want to get win-loss ratios
        years = [2022, 2023]

        # Initialize dictionaries to store win and loss counts for each year
        win_counts = {str(year): 0 for year in years}
        loss_counts = {str(year): 0 for year in years}

        for year in years:
            starttime = int(datetime(year, 1, 1).timestamp())
            endtime = int(datetime(year, 12, 31, 23, 59, 59).timestamp())

            # Initialize variables
            wins = 0
            losses = 0
            start_index = 0
            count_per_request = 100

            while True:
                # Set parameters to retrieve ranked matches starting from index 'start_index' and up to 'count_per_request' matches
                matchlist = lol_watcher.match.matchlist_by_puuid(
                    row['REGION'],
                    row['PUUID'],
                    queue=420,
                    start_time=starttime,
                    end_time=endtime,
                    start=start_index,
                    count=count_per_request
                )

                # Loop through matches and participants to count wins and losses
                for match_id in matchlist:
                    try:
                        match_details = lol_watcher.match.by_id(row['REGION'], match_id)

                        for participant_identity in match_details['info']['participants']:
                            if participant_identity['puuid'] == row['PUUID']:
                                win = participant_identity['win']

                                if win is not None:
                                    if win:
                                        wins += 1
                                    else:
                                        losses += 1
                    except Exception as match_error:
                        print(f"Error fetching match details for match ID {match_id}: {match_error}")

                # Break the loop if there are fewer than 'count_per_request' matches returned
                if len(matchlist) < count_per_request:
                    break

                # Increment start_index for the next batch
                start_index += count_per_request

            # Update win and loss counts for the current year
            win_counts[str(year)] = wins
            loss_counts[str(year)] = losses

        # Calculate win-loss ratios for each year
        ratios = {year: wins / (wins + losses) if (wins + losses) > 0 else 0 for year, wins, losses in zip(years, win_counts.values(), loss_counts.values())}

        # Print or use the win-loss ratios for each year
        print(f"Row {row.name + 1}: {row['NICKNAME']}, Total Rows: {len(df_new)}, Win-Loss Ratios: {ratios}")

        # Return a pandas Series with the win-loss ratios for each year
        return pd.Series(ratios)

    except Exception as e:
        print(f"Error fetching matchlist for {row['NICKNAME']}: {e}")
        return pd.Series({str(year): 0 for year in years})

In [ ]:
def get_avg_game_duration(row):
    try:
        # Define the years for which you want to get match durations
        years = [2022, 2023]

        # Initialize dictionaries to store total duration and game counts for each year
        total_durations = {str(year): 0 for year in years}
        game_counts = {str(year): 0 for year in years}

        for year in years:
            starttime = int(datetime(year, 1, 1).timestamp())
            endtime = int(datetime(year, 12, 31, 23, 59, 59).timestamp())

            # Initialize variables
            total_duration = 0
            game_count = 0
            start_index = 0
            count_per_request = 100

            while True:
                # Set parameters to retrieve ranked matches starting from index 'start_index' and up to 'count_per_request' matches
                matchlist = lol_watcher.match.matchlist_by_puuid(
                    row['REGION'],
                    row['PUUID'],
                    queue=420,  # Ranked Solo/Duo queue
                    start_time=starttime,
                    end_time=endtime,
                    start=start_index,
                    count=count_per_request
                )

                # Loop through matches and accumulate the duration time and count games
                for match_id in matchlist:
                    try:
                        match_details = lol_watcher.match.by_id(row['REGION'], match_id)

                        # Get the duration of the match in seconds
                        game_duration = match_details['info']['gameDuration']

                        # Check if game duration is not NaN and not None
                        if not pd.isnull(game_duration) and game_duration is not None:
                            total_duration += game_duration
                            game_count += 1

                            # Print the game duration for each nickname
                            #print(f"{row['NICKNAME']} - {game_duration} seconds in {game_count} games")

                    except Exception as match_error:
                        print(f"Error fetching match details for match ID {match_id}: {match_error}")

                # Break the loop if there are fewer than 'count_per_request' matches returned
                if len(matchlist) < count_per_request:
                    break

                # Increment start_index for the next batch
                start_index += count_per_request

            # Update the total duration and game count for the current year
            total_durations[str(year)] = total_duration
            game_counts[str(year)] = game_count

        # Calculate average game durations for each year, handling NaN values
        print({year: total_durations[str(year)] / game_counts[str(year)] if game_counts[str(year)] > 0 else 0 for year in years})
        avg_durations = {year: total_durations[str(year)] / game_counts[str(year)] if game_counts[str(year)] > 0 else 0 for year in years}
        print(f"{row['NICKNAME']} - {avg_durations[str(year)]} seconds in {game_counts[str(year)]}")

        # Return a pandas Series with the average durations for each year
        return pd.Series({
            'avg_duration_2022': avg_durations[str(2022)],
            'avg_duration_2023': avg_durations[str(2023)]
        })

    except Exception as e:
        return pd.Series({
            'avg_duration_2022': total_durations[str(2022)] / game_counts[str(2022)] if game_counts[str(2022)] > 0 else 0,
            'avg_duration_2023': total_durations[str(2023)] / game_counts[str(2023)] if game_counts[str(2023)] > 0 else 0
        })

In [ ]:
# Function to get the total match count for a given player from 2021 to 2023 of other types of games:
def get_match_counts(row):
    try:
        years = [2021, 2022, 2023]
        queues = [400, 430, 440, 450] #queue types in 'https://static.developer.riotgames.com/docs/lol/queues.json'
        match_counts = {str(year): {str(queue): 0 for queue in queues} for year in years}

        def fetch_match_list(year, queue, start_index, count_per_request):
            starttime = int(datetime(year, 1, 1).timestamp())
            endtime = int(datetime(year, 12, 31, 23, 59, 59).timestamp())
            total_matches = 0
            start_index = 0
            count_per_request = 100
            while True:
                matchlist = lol_watcher.match.matchlist_by_puuid(
                    row['REGION'],
                    row['PUUID'],
                    queue=queue,
                    start_time=starttime,
                    end_time=endtime,
                    start=start_index,
                    count=count_per_request
                )
                total_matches += len(matchlist)
                if len(matchlist) < count_per_request:
                    break
                start_index += count_per_request
            return total_matches

        for year in years:
            for queue in queues:
                match_counts[str(year)][str(queue)] = fetch_match_list(year, queue, 0, 100)

        match_counts_flat = {f'{year}_Queue_{queue}': count for year, counts in match_counts.items() for queue, count in counts.items()}

        print(f"Row {row.name + 1}: {row['NICKNAME']}, Total Rows: {len(df_new)}, Match Counts: {match_counts_flat}")
        return pd.Series(match_counts_flat)

    except Exception as e:
        print(f"Error fetching matchlist for {row['NICKNAME']}: {e}")
        return pd.Series({f"{year}_Queue_{queue}": 0 for year in years for queue in queues})

In [ ]:
def get_most_played_champion(row):
    try:
        years = [2022, 2023]
        champion_counts = {str(year): {} for year in years}

        for year in years:
            starttime = int(datetime(year, 1, 1).timestamp())
            endtime = int(datetime(year, 12, 31, 23, 59, 59).timestamp())

            start_index = 0
            count_per_request = 100

            while True:
                matchlist = lol_watcher.match.matchlist_by_puuid(
                    row['REGION'],
                    row['PUUID'],
                    queue=420,
                    start_time=starttime,
                    end_time=endtime,
                    start=start_index,
                    count=count_per_request
                )

                for match_id in matchlist:
                    try:
                        match_details = lol_watcher.match.by_id(row['REGION'], match_id)

                        for participant in match_details['info']['participants']:
                            if participant['puuid'] == row['PUUID']:
                                champion_name = participant['championName']
                                champion_counts[str(year)][champion_name] = champion_counts[str(year)].get(champion_name, 0) + 1
                                break

                    except Exception as match_error:
                        print(f"Error fetching match details for match ID {match_id}: {match_error}")

                if len(matchlist) < count_per_request:
                    break

                start_index += count_per_request

        most_played_champions = {
            str(year): max(champion_counts[str(year)], key=champion_counts[str(year)].get, default=None) for year in years
        }

        return pd.Series({
            'most_played_champ_2022': most_played_champions.get("2022", None),
            'most_played_champ_2023': most_played_champions.get("2023", None)
        })

    except Exception as e:
        return pd.Series({
            'most_played_champ_2022': None,
            'most_played_champ_2023': None
        })

In [ ]:
def get_most_played_position(row):
    try:
        years = [2022, 2023]
        position_counts = {str(year): {} for year in years}
        most_played_champions = {
            "2022": row.get('most_played_champ_2022'),
            "2023": row.get('most_played_champ_2023'),
        }

        for year in years:
            if not most_played_champions[str(year)]:
                continue  # Skip if there's no most played champion

            starttime = int(datetime(year, 1, 1).timestamp())
            endtime = int(datetime(year, 12, 31, 23, 59, 59).timestamp())

            start_index = 0
            count_per_request = 100

            while True:
                matchlist = lol_watcher.match.matchlist_by_puuid(
                    row['REGION'],
                    row['PUUID'],
                    queue=420,
                    start_time=starttime,
                    end_time=endtime,
                    start=start_index,
                    count=count_per_request
                )

                for match_id in matchlist:
                    try:
                        match_details = lol_watcher.match.by_id(row['REGION'], match_id)

                        for participant in match_details['info']['participants']:
                            if participant['puuid'] == row['PUUID'] and participant['championName'] == most_played_champions[str(year)]:
                                role = participant['teamPosition'].lower()
                                if role in ['top', 'jungle', 'middle', 'bottom', 'utility']:
                                    role = 'mid' if role == 'middle' else ('adc' if role == 'bottom' else role)
                                    position_counts[str(year)][role] = position_counts[str(year)].get(role, 0) + 1
                                break

                    except Exception as match_error:
                        print(f"Error fetching match details for match ID {match_id}: {match_error}")

                if len(matchlist) < count_per_request:
                    break

                start_index += count_per_request

        most_played_positions = {
            str(year): max(position_counts[str(year)], key=position_counts[str(year)].get, default=None) for year in years
        }

        return pd.Series({
            'most_played_position_2022': most_played_positions.get("2022", None),
            'most_played_position_2023': most_played_positions.get("2023", None)
        })

    except Exception as e:
        return pd.Series({
            'most_played_position_2022': None,
            'most_played_position_2023': None
        })

In [ ]:
def calculate_frustration_kpi(row):
    try:
        year = 2023  # Only calculating for 2023
        frustration_scores = []  # Track frustration scores per match
        total_games = 0  # Track number of games played

        starttime = int(datetime(year, 1, 1).timestamp())
        endtime = int(datetime(year, 12, 31, 23, 59, 59).timestamp())
        start_index = 0
        count_per_request = 100

        while True:
            matchlist = lol_watcher.match.matchlist_by_puuid(
                row['REGION'], row['PUUID'], queue=420,
                start_time=starttime, end_time=endtime,
                start=start_index, count=count_per_request
            )

            for match_id in matchlist:
                try:
                    match_details = lol_watcher.match.by_id(row['REGION'], match_id)
                    match_frustration = 0

                    for participant in match_details['info']['participants']:
                        if participant['puuid'] == row['PUUID']:
                            # Game result (Loss = +1 point)
                            if not participant['win']:
                                match_frustration += 1

                            # KDA < 1 & Deaths > 10 (+2 points)
                            kda = (participant['kills'] + participant['assists']) / max(1, participant['deaths'])
                            if kda < 1 and participant['deaths'] > 10:
                                match_frustration += 2

                            # Early surrender (<20 min) (+1 point)
                            if 'gameEndTimestamp' in match_details['info'] and 'gameStartTimestamp' in match_details['info']:
                                game_duration = (match_details['info']['gameEndTimestamp'] - match_details['info']['gameStartTimestamp']) / 1000
                                if game_duration < 1200:
                                    match_frustration += 1

                            # AFK / Disconnect (+2 points) ## Might be more times internet issues related, not frustration...
                            #if participant.get('gameEndedInEarlySurrender', False) or participant.get('detached', False):
                                #match_frustration += 2

                            # Deaths in short time (<2 min) (+1 point)
                            death_timestamps = [
                                event['timestamp'] for event in match_details['info'].get('events', [])
                                if event['type'] == 'CHAMPION_KILL' and event['victimId'] == participant['participantId']
                            ]
                            for i in range(1, len(death_timestamps)):
                                if (death_timestamps[i] - death_timestamps[i-1]) / 1000 < 120:
                                    match_frustration += 1
                                    break

                            # Panic Flashing (+1 point if 3+ useless Flashes)
                            flash_uses = 0
                            useless_flashes = 0
                            for event in match_details['info'].get('events', []):
                                if event['type'] == 'ITEM_PURCHASED' and event['participantId'] == participant['participantId'] and event['itemId'] == 4:  # Flash ID = 4
                                    flash_uses += 1
                                    # Check if kill/assist happened within 5 sec of Flash
                                    flash_time = event['timestamp']
                                    kill_or_assist = any(
                                        e for e in match_details['info'].get('events', [])
                                        if e['type'] == 'CHAMPION_KILL' and (
                                            e.get('killerId') == participant['participantId'] or
                                            participant['participantId'] in e.get('assistingParticipantIds', [])
                                        ) and 0 <= (e['timestamp'] - flash_time) / 1000 <= 5
                                    )
                                    if not kill_or_assist:
                                        useless_flashes += 1
                            if useless_flashes >= 3:
                                match_frustration += 1

                        # Check if any other participant has KDA < 1 & Deaths > 10 (+2 points)
                        if participant['puuid'] != row['PUUID']:
                            kda_other = (participant['kills'] + participant['assists']) / max(1, participant['deaths'])
                            if kda_other < 1 and participant['deaths'] > 10:
                                match_frustration += 2

                    frustration_scores.append(match_frustration)  # Add match frustration score to the list
                    total_games += 1  # Increment the total games for 2023
                except Exception as match_error:
                    print(f"Error fetching match details for match ID {match_id}: {match_error}")

            if len(matchlist) < count_per_request:
                break
            start_index += count_per_request

        # Calculate stats for 2023
        if frustration_scores:  # Check if there are any scores
            avg_kpi = sum(frustration_scores) / len(frustration_scores)
            min_kpi = min(frustration_scores)
            max_kpi = max(frustration_scores)
            frustration_pct = (sum(1 for score in frustration_scores if score >= 3) / len(frustration_scores)) * 100
            frustration_count = sum(1 for score in frustration_scores if score >= 3)
            ngames = len(frustration_scores)
        else:
            avg_kpi, min_kpi, max_kpi, frustration_pct, frustration_count, ngames = 0, 0, 0, 0, 0, 0

        # Return values as a flattened pd.Series
        return pd.Series({
            '2023_avg_kpi': avg_kpi,
            '2023_min_kpi': min_kpi,
            '2023_max_kpi': max_kpi,
            '2023_frustration_pct': frustration_pct, # % of ranked games with frustration KPI >=3
            '2023_frustration_ngames': frustration_count, # total number of ranked games with frustration KPI >=3
            '2023_ngames': ngames #total number of ranked games for the analysis
        })

    except Exception as e:
        print(f"Error calculating frustration KPI for {row['NICKNAME']}: {e}")
        return pd.Series({
            '2023_avg_kpi': 0,
            '2023_min_kpi': 0,
            '2023_max_kpi': 0,
            '2023_frustration_pct': 0,
            '2023_frustration_ngames': 0,
            '2023_ngames': 0
        })


In [ ]:
df_new = df.copy()
#df_new['REGION'] = df_new['REGION'].replace({'LA1': 'LAN', 'LA2': 'LAS'}) #POTSER ÉS NECESSARI, ja ho comentem

In [ ]:
#df_new = df[1:7].copy()
#df_new['REGION'] = df_new['REGION'].replace({'LA1': 'LAN', 'LA2': 'LAS'})

In [ ]:
df_new.head()

In [ ]:
df_new[['RANKED_WIN_RATIO_2022', 'RANKED_WIN_RATIO_2023']] = df_new.apply(get_win_loss_ratios, axis=1)

In [ ]:
df_new[['RANKED_AVG_MATCH_DURATION_2022', 'RANKED_AVG_MATCH_DURATION_2023']] = df_new.apply(get_avg_game_duration, axis=1)

In [ ]:
df_new[['2021_400_COUNT', '2022_400_COUNT', '2023_400_COUNT',
    '2021_430_COUNT', '2022_430_COUNT', '2023_430_COUNT',
    '2021_440_COUNT', '2022_440_COUNT', '2023_440_COUNT',
    '2021_450_COUNT', '2022_450_COUNT', '2023_450_COUNT']] = df_new.apply(get_match_counts, axis=1)

In [ ]:
df_new[['most_played_champ_2022','most_played_champ_2023']] = df_new.apply(get_most_played_champion, axis=1)

In [ ]:
df_new[['most_played_position_2022','most_played_position_2023']] = df_new.apply(get_most_played_position, axis=1)

In [ ]:
# KPI:
df_new[['2023_avg_kpi', '2023_min_kpi', '2023_max_kpi', '2023_frustration_pct', '2023_frustration_ngames', '2023_ngames']] = df_new.apply(calculate_frustration_kpi, axis=1)

In [ ]:
df_new.head()

In [ ]:
file_path_to_extract = '' #insert the file path where you want to save the final file
df_new.to_excel(file_path_to_extract)